In [1]:
import numpy as np


In [ ]:

def secant_method(x1,x2,df_x1,df_x2):
    a = (df_x2 - df_x1)/(x2 - x1)
    b = df_x1 - x1 * a
    return -b/a

def brent(f,df,start,end,eps=1e-5):
    u = x = w = v = 0.5 * (start + end)
    f_u = f_x = f_w = f_v = f(x)
    df_u = df_x = df_w = df_v = df(x)
    d = e = end - start

    function_calls_number = 1
    step_details = []

    while True:
        step_details.append([start, end, x, f_x, df_x, w, f_w, df_w, v, f_v, df_v, d, e, u, f_u, df_u])
        g,e = e,d
        candidates = []

        if x != w and df_x != df_w:
            u1 = secant_method(x,w,df_x,df_w)
            if start + eps <= u1 <= end - eps and np.abs(u1 - x) < 0.5 * g:
                candidates.append(u1)

        if x != v and df_x != df_v:
            u2 = secant_method(x,v,df_x,df_v)
            if start + eps <= u2 <= end - eps and np.abs(u2 - x) < 0.5 * g:
                candidates.append(u2)

        if len(candidates) == 2:
            u = candidates[0]
        else:
            if df_x > 0:
                u = 0.5 * (start + x)
            else:
                u = 0.5 * (x + end)

        if np.abs(u - x) < eps:
            u = x + np.sign(u - x) * eps

        d = np.abs(x - u)
        f_u = f(u)
        df_u = df(u)
        function_calls_number += 1
        if f_u <= f_x:
            if u >= x:
                start = x
            else:
                end = x
            v,w,x = w,x,u
            f_v,f_w,f_x = f_w,f_x,f_u
            df_v,df_w,df_x = df_w,df_x,df_u
        else:
            if u >= x:
                end = u
            else:
                start = u
            if f_u <= f_w or w == x:
                v,w = w,u
                f_v,f_w = f_w,f_u
                df_v,df_w = df_w,df_u
            elif f_u <= f_v or v == x or v == w:
                v = u
                f_v = f_u
                df_v = df_u

            if abs(d - g) > eps and end - start != 0:
                break

    step_details.append([start, end, x, f_x, df_x, w, f_w, df_w, v, f_v, df_v, d, e])
    return x, function_calls_number, step_details


In [ ]:
def coordinate_descent(X0,f,df,eps=1e-5,max_iter=5e4):
    step_details = []
    args_count = len(X0)
    alpha = 0 #TODO:???
    k = 0

    X = X0.copy()
    step_details.append([X, None, f(X)])

    while k < max_iter:
        i = k % args_count
        next_X = X.copy()
        next_X[i] -= alpha*df(X, i)

        step_details.append([next_X, X, f(X)])
        prev_X, X = X, next_X
        k += 1

        if np.sum(np.abs(step_details[-1][-1] - step_details[-2][-1])) < eps:
            break

    return X, step_details


In [ ]:

def golden_ratio(function,start,end,epsilon=1e-5):
    phi = 0.5 * (3 - np.sqrt(5))

    x1 = start + (end - start)*phi
    x2 = end - (end - start)*phi

    x1_value = function(x1)
    x2_value = function(x2)

    while end - start > epsilon:
        if x1_value > x2_value:
            start = x1
            x1 = x2
            x2 = end + start - x1
            x1_value = x2_value
            x2_value = function(x2)
        else:
            end = x2
            x2 = x1
            x1 = start + end - x2
            x2_value = x1_value
            x1_value = function(x1)

    return 0.5 * (x1 + x2)


def steepest_descent(X0,f,df,eps=1e-5,max_iter=5e4):
    step_details = []
    args_count = len(X0)
    k = 0

    X = X0.copy()

    while k < max_iter:
        g = np.array([df(X, i) for i in range(args_count)])
        f_next_X = lambda a: X - a * g
        alpha = golden_ratio(lambda a: f(f_next_X(a)), -10, 10, epsilon=eps)
        next_X = f_next_X(alpha)

        step_details.append([next_X, X, f(X)])
        prev_X, X = X, next_X
        k += 1

        if np.sum(np.abs(step_details[-1][-1] - step_details[-2][-1])) < eps:
            break

    return X, step_details
